In [1]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
from scipy.stats import trim_mean
from sklearn.metrics import mean_squared_error

sys.path.append('..')
from higgs_inference import settings
from higgs_inference.various.utils import format_number

In [2]:
result_dir = '../results/'

# TablePrinter class

In [3]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []
        self.content_in_last_block = False

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Skip entirely empty lines
            try:
                if not np.any(np.isfinite(line[2:])):
                    continue
            except TypeError:
                print(line)
                continue
                
            self.content_in_last_block = True
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j])) + ')'
                    else:
                        text += format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        if self.content_in_last_block:
            self.table += self.indent + self.midrule + self.end_line
            self.content_in_last_block = False
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            bracket = False
            try:
                value = fn(filename, folder)
            except (IOError, ValueError):
                #print('File', filename, 'in folder', folder, 'not found')
                value = np.nan
                
            if isinstance(value, (list, tuple)):
                value, bracket = value
                
            line.append(value)
            brackets.append(bracket)

        self.block_entries.append(line)
        self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [4]:
def expected_mse_log_r(filename, folder='parameterized'):
    mse_log_r = np.load(result_dir + folder + '/mse_logr_' + filename + '.npy')[settings.thetas_train]
    return np.mean((mse_log_r))
                       
def expected_trimmed_mse_log_r(filename, folder='parameterized'):
    mse_log_r = np.load(result_dir + folder + '/trimmed_mse_logr_' + filename + '.npy')[settings.thetas_train]
    return np.mean((mse_log_r))

In [5]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.sqrt(np.median(errors))

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.median(errors)

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return np.sqrt(trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return np.sqrt(trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

# Main tables

In [6]:
labels = ['Histogram',
           r'AFC', # (density est.\ in $\boldx$)',
          'carl (PbP, raw)',
          'carl (PbP, cal.)',
          'carl (param., raw)',
          'carl (param., cal.)',
          'carl (aware, raw)',
          'carl (aware, cal.)',
          
          'SM score regr.\ + density est.',
          'carl + score (param., raw)',
          'carl + score (param., cal.)',
          'carl + score (aware, raw)',
          'carl + score (aware, cal.)',
          
          'Ratio regression (PbP, raw)',
          'Ratio regression (PbP, cal.)',
          'Ratio regression (param., raw)',
          'Ratio regression (param., cal.)',
          'Ratio regression (aware, raw)',
          'Ratio regression (aware, cal.)',
          
          r'Ratio + score regr.\ (param., raw)',
          r'Ratio + score regr.\ (param., cal.)',
          r'Ratio + score regr.\ (aware, raw)',
          r'Ratio + score regr.\ (aware, cal.)']

folders = (['histo'] + ['afc'] + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['score_regression'] + ['parameterized'] * 4
          + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['parameterized'] * 4)

filenames = ['histo',
             'afc',
             'carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'carl_aware',
             'carl_calibrated_aware',
             
             'scoreregression',
             'combined',
             'combined_calibrated',
             'combined_aware',
             'combined_calibrated_aware',
             
             'regression',
             'regression_calibrated',
             'regression',
             'regression_calibrated',
             'regression_aware',
             'regression_calibrated_aware',
             
             'combinedregression',
             'combinedregression_calibrated'
             'combinedregression_aware',
             'combinedregression_calibrated_aware']


def show_main_table(algorithm_begin=0, algorithm_end=None):
    
    #table = TablePrinter([trimmed_mse_log_r_nottrained, mse_expected_log_r],
    #                      precisions=[3,2])
    table = TablePrinter([expected_mse_log_r, expected_trimmed_mse_log_r],
                          precisions=[4,4])

    for i, (label, filename, folder) in enumerate(
        zip(labels[algorithm_begin:algorithm_end], filenames[algorithm_begin:algorithm_end], folders[algorithm_begin:algorithm_end])):

        if i > 0:
            table.new_block()

        if folder == 'point_by_point':
            table.add('', 'PbP', filename, 'point_by_point')
            table.add('', 'PbP, shallow', filename + '_shallow', 'point_by_point')
            table.add('', 'PbP, deep', filename + '_deep', 'point_by_point')

        elif folder == 'histo':
            table.add(label, r'$p_{T,j1}$', filename + '_ptj', 'histo')
            table.add('', r'$\Delta \phi_{jj}$', filename + '_deltaphi', 'histo')
            table.add('', '2d', filename + '_2d', 'histo')

        elif folder == 'afc':
            table.add(label, '2d, $\epsilon = 1$', filename + '_2d_epsilon_1.00', 'afc')
            table.add('', '2d, $\epsilon = 0.5$', filename + '_2d_epsilon_0.50', 'afc')
            table.add('', '2d, $\epsilon = 0.2$', filename + '_2d_epsilon_0.20', 'afc')
            table.add('', '2d, $\epsilon = 0.1$', filename + '_2d_epsilon_0.10', 'afc')
            table.add('', '2d, $\epsilon = 0.05$', filename + '_2d_epsilon_0.05', 'afc')
            table.add('', '2d, $\epsilon = 0.02$', filename + '_2d_epsilon_0.02', 'afc')
            table.add('', '2d, $\epsilon = 0.01$', filename + '_2d_epsilon_0.01', 'afc')

            table.add(label, '5d, $\epsilon = 1$', filename + '_5d_epsilon_1.00', 'afc')
            table.add('', '5d, $\epsilon = 0.5$', filename + '_5d_epsilon_0.50', 'afc')
            table.add('', '5d, $\epsilon = 0.2$', filename + '_5d_epsilon_0.20', 'afc')
            table.add('', '5d, $\epsilon = 0.1$', filename + '_5d_epsilon_0.10', 'afc')
            table.add('', '5d, $\epsilon = 0.05$', filename + '_5d_epsilon_0.05', 'afc')
            table.add('', '5d, $\epsilon = 0.02$', filename + '_5d_epsilon_0.02', 'afc')
            table.add('', '5d, $\epsilon = 0.01$', filename + '_5d_epsilon_0.01', 'afc')

        elif folder == 'score_regression':
            table.add(label, r'Fixed density est. on $t$', filename + '_score', 'score_regression')
            table.add('', r'Dyn. dens. est. on $t$', filename + '_rotatedscore', 'score_regression')
            table.add('', r'Dens. est. on $t \cdot \theta$', filename + '_scoretheta', 'score_regression')

        else:
            table.add(label, r'Baseline', filename)
            table.add('', r'Baseline, shallow', filename + '_shallow')
            table.add('', r'Baseline, deep', filename + '_deep')
            
            table.add('', r'Baseline, $\alpha = 0.01$', filename + '_alpha_0.010')
            table.add('', r'Baseline, $\alpha = 0.02$', filename + '_alpha_0.020')
            table.add('', r'Baseline, $\alpha = 0.05$', filename + '_alpha_0.050')
            table.add('', r'Baseline, $\alpha = 0.1$', filename + '_alpha_0.10')
            table.add('', r'Baseline, $\alpha = 0.2$', filename + '_alpha_0.20')
            table.add('', r'Baseline, $\alpha = 0.5$', filename + '_alpha_0.50')
            table.add('', r'Baseline, $\alpha = 1$', filename + '_alpha_1.0')
            table.add('', r'Baseline, $\alpha = 2$', filename + '_alpha_2.0')
            table.add('', r'Baseline, $\alpha = 5$', filename + '_alpha_5.0')
            table.add('', r'Baseline, $\alpha = 10$', filename + '_alpha_10')
            table.add('', r'Baseline, $\alpha = 20$', filename + '_alpha_20')
            table.add('', r'Baseline, $\alpha = 50$', filename + '_alpha_50')
            table.add('', r'Baseline, $\alpha = 100$', filename + '_alpha_100')
            
            table.add('', r'Baseline, large batches', filename + '_largebatch')
            table.add('', r'Baseline, small batches', filename + '_smallbatch')
            table.add('', r'Baseline, const.\ LR', filename + '_constantlr')
            table.add('', r'Baseline, const.\ LR, large batches', filename + '_constantlr_largebatch')
            table.add('', r'Baseline, const.\ LR, small batches', filename + '_constantlr_largebatch')
            table.add('', r'Baseline, small LR', filename + '_slowlearning')
            table.add('', r'Baseline, small LR, large batches', filename + '_slowlearning_largebatch')
            table.add('', r'Baseline, small LR, small batches', filename + '_slowlearning_smallbatch')
            table.add('', r'Baseline, small const.\ LR', filename + '_slowlearning_constantlr')
            table.add('', r'Baseline, small const.\ LR, large batches', filename + '_slowlearning_constantlr_largebatch')
            table.add('', r'Baseline, small const.\ LR, small batches', filename + '_slowlearning_constantlr_smallbatch')
            table.add('', r'Baseline, large LR', filename + '_fastlearning')
            table.add('', r'Baseline, large LR, large batches', filename + '_fastlearning_largebatch')
            table.add('', r'Baseline, large LR, small batches', filename + '_fastlearning_smallbatch')
            table.add('', r'Baseline, large const.\ LR', filename + '_fastlearning_constantlr')
            table.add('', r'Baseline, large const.\ LR, large batches', filename + '_fastlearning_constantlr_largebatch')
            table.add('', r'Baseline, large const.\ LR, small batches', filename + '_fastlearning_constantlr_smallbatch')
            
            table.add('', r'Random $\boldtheta$, shallow', filename + '_random_shallow')
            table.add('', r'Random $\boldtheta$', filename + '_random')
            table.add('', r'Random $\boldtheta$, deep', filename + '_random_deep')
            
            table.add('', r'Morphing basis, shallow', filename + '_basis_shallow')
            table.add('', r'Morphing basis', filename + '_basis')
            table.add('', r'Morphing basis, deep', filename + '_basis_deep')

    print(table.print())

In [7]:
show_main_table(0,7)

   Histogram & $p_{T,j1}$ & 0.2633 & 0.0814\\
    & $\Delta \phi_{jj}$ & 0.3233 & 0.0991\\
    & 2d & \emph{0.1545} & \emph{0.0328}\\
   \midrule
   AFC & 2d, $\epsilon = 1$ & 0.2883 & 0.0757\\
   AFC & 5d, $\epsilon = 1$ & 0.2746 & 0.0677\\
    & 5d, $\epsilon = 0.5$ & \emph{0.1866} & \emph{0.0376}\\
    & 5d, $\epsilon = 0.2$ & 0.2168 & 0.0565\\
    & 5d, $\epsilon = 0.1$ & 2.7947 & 0.8854\\
    & 5d, $\epsilon = 0.05$ & 73.8664 & 13.2071\\
    & 5d, $\epsilon = 0.02$ & 6046.7529 & 240.5334\\
    & 5d, $\epsilon = 0.01$ & 151909.8097 & 16891.7454\\
   \midrule
   carl (param., raw) & Baseline & 0.0288 & 0.0069\\
    & Baseline, shallow & \emph{0.0254} & \emph{0.0060}\\
    & Baseline, deep & 0.0309 & 0.0067\\
    & Random $\boldtheta$, shallow & 0.0275 & 0.0063\\
    & Random $\boldtheta$ & 0.0275 & 0.0064\\
   \midrule
   carl (param., cal.) & Baseline & 0.0298 & 0.0060\\
    & Baseline, shallow & \emph{0.0245} & \emph{0.0053}\\
    & Baseline, deep & 0.0325 & 0.0059\\
    & Random 

In [8]:
show_main_table(7,12)

   SM score regr.\ + density est. & Fixed density est. on $t$ & \emph{0.0311} & \emph{0.0019}\\
    & Dyn. dens. est. on $t$ & 0.0313 & 0.0020\\
    & Dens. est. on $t \cdot \theta$ & 0.0585 & 0.0038\\
   \midrule
   carl + score (param., raw) & Baseline & 0.0034 & 0.0007\\
    & Baseline, shallow & 0.0065 & 0.0009\\
    & Baseline, deep & \emph{0.0026} & \emph{0.0007}\\
    & Baseline, $\alpha = 0.1$ & 0.0063 & 0.0014\\
    & Baseline, $\alpha = 0.2$ & 0.0048 & 0.0011\\
    & Baseline, $\alpha = 0.5$ & 0.0038 & 0.0008\\
    & Random $\boldtheta$, shallow & 0.0068 & 0.0009\\
    & Random $\boldtheta$ & 0.0035 & 0.0007\\
   \midrule
   carl + score (param., cal.) & Baseline & \emph{0.0027} & \emph{0.0006}\\
    & Baseline, shallow & 0.0051 & 0.0007\\
    & Baseline, $\alpha = 0.1$ & 0.0061 & 0.0014\\
    & Baseline, $\alpha = 0.2$ & 0.0044 & 0.0009\\
    & Random $\boldtheta$, shallow & 0.0055 & 0.0007\\
    & Random $\boldtheta$ & 0.0028 & 0.0006\\
   \midrule



In [9]:
show_main_table(12,18)

   Ratio regression (param., raw) & Baseline & \emph{0.0055} & 0.0030\\
    & Baseline, shallow & 0.0098 & 0.0052\\
    & Random $\boldtheta$ & 0.0055 & \emph{0.0030}\\
   \midrule
   Ratio regression (param., cal.) & Baseline & \emph{0.0052} & 0.0028\\
    & Baseline, shallow & 0.0090 & 0.0047\\
    & Random $\boldtheta$ & 0.0053 & \emph{0.0028}\\
   \midrule



In [10]:
show_main_table(18,None)

   Ratio + score regr.\ (param., raw) & Baseline & 0.0018 & 0.0007\\
    & Baseline, shallow & 0.0044 & 0.0012\\
    & Baseline, $\alpha = 5$ & 0.0022 & 0.0009\\
    & Baseline, $\alpha = 10$ & 0.0019 & 0.0008\\
    & Baseline, $\alpha = 50$ & \emph{0.0017} & \emph{0.0007}\\
    & Random $\boldtheta$ & 0.0018 & 0.0007\\
   \midrule



# Score accuracy table

In [30]:
labels = ['carl (param.)',
          'carl (aware)',
          'carl + score regr.\ (param.)',
          'carl + score regr.\ (aware)',
          'Ratio regression (param.)',
          'Ratio regression (aware)',
          'Ratio + score regr.\ (param.)',
          'Ratio + score regr.\ (aware)']

filenames = ['carl',
             'carl_aware',
             'combined',
             'combined_aware',
             'regression',
             'regression_aware',
             'combinedregression',
             'combinedregression_aware']

table = TablePrinter([trimmed_mse_score_nottrained],
                      precisions=[2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table.new_block()
        
        table.add(label, r'Baseline, shallow', filename + '_shallow')
        table.add(label, r'Baseline', filename)
        table.add('', r'Baseline, deep', filename + '_deep')

        table.add('', r'Baseline, $\alpha = 0.01$', filename + '_alpha_0.010')
        table.add('', r'Baseline, $\alpha = 0.02$', filename + '_alpha_0.020')
        table.add('', r'Baseline, $\alpha = 0.05$', filename + '_alpha_0.050')
        table.add('', r'Baseline, $\alpha = 0.1$', filename + '_alpha_0.10')
        table.add('', r'Baseline, $\alpha = 0.2$', filename + '_alpha_0.20')
        table.add('', r'Baseline, $\alpha = 0.5$', filename + '_alpha_0.50')
        table.add('', r'Baseline, $\alpha = 1$', filename + '_alpha_1.0')
        table.add('', r'Baseline, $\alpha = 2$', filename + '_alpha_2.0')
        table.add('', r'Baseline, $\alpha = 5$', filename + '_alpha_5.0')
        table.add('', r'Baseline, $\alpha = 10$', filename + '_alpha_10')
        table.add('', r'Baseline, $\alpha = 20$', filename + '_alpha_20')
        table.add('', r'Baseline, $\alpha = 50$', filename + '_alpha_50')
        table.add('', r'Baseline, $\alpha = 100$', filename + '_alpha_100')

        table.add('', r'Baseline, large batches', filename + '_largebatch')
        table.add('', r'Baseline, small batches', filename + '_smallbatch')
        table.add('', r'Baseline, const.\ LR', filename + '_constantlr')
        table.add('', r'Baseline, const.\ LR, large batches', filename + '_constantlr_largebatch')
        table.add('', r'Baseline, const.\ LR, small batches', filename + '_constantlr_largebatch')
        table.add('', r'Baseline, small LR', filename + '_slowlearning')
        table.add('', r'Baseline, small LR, large batches', filename + '_slowlearning_largebatch')
        table.add('', r'Baseline, small LR, small batches', filename + '_slowlearning_smallbatch')
        table.add('', r'Baseline, small const.\ LR', filename + '_slowlearning_constantlr')
        table.add('', r'Baseline, small const.\ LR, large batches', filename + '_slowlearning_constantlr_largebatch')
        table.add('', r'Baseline, small const.\ LR, small batches', filename + '_slowlearning_constantlr_smallbatch')
        table.add('', r'Baseline, large LR', filename + '_fastlearning')
        table.add('', r'Baseline, large LR, large batches', filename + '_fastlearning_largebatch')
        table.add('', r'Baseline, large LR, small batches', filename + '_fastlearning_smallbatch')
        table.add('', r'Baseline, large const.\ LR', filename + '_fastlearning_constantlr')
        table.add('', r'Baseline, large const.\ LR, large batches', filename + '_fastlearning_constantlr_largebatch')
        table.add('', r'Baseline, large const.\ LR, small batches', filename + '_fastlearning_constantlr_smallbatch')

        table.add('', r'Random $\boldtheta$, shallow', filename + '_random_shallow')
        table.add('', r'Random $\boldtheta$', filename + '_random')
        table.add('', r'Random $\boldtheta$, deep', filename + '_random_deep')

        table.add('', r'Morphing basis, shallow', filename + '_basis_shallow')
        table.add('', r'Morphing basis', filename + '_basis')
        table.add('', r'Morphing basis, deep', filename + '_basis_deep')

print(table.print())

IndexError: too many indices for array